## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-05-16-53-53 +0000,nyt,Venezuela Live Updates: Maduro Arrives at Cour...,https://www.nytimes.com/live/2026/01/05/world/...
1,2026-01-05-16-53-36 +0000,bbc,Only Greenland and Denmark should decide its f...,https://www.bbc.com/news/articles/cy9yq8znq37o...
2,2026-01-05-16-53-16 +0000,wapo,Live updates: Venezuela’s Nicolás Maduro arriv...,https://www.washingtonpost.com/world/2026/01/0...
3,2026-01-05-16-53-16 +0000,wapo,U.S. defends Venezuela attacks at U.N. meeting,https://www.washingtonpost.com
4,2026-01-05-16-52-03 +0000,bbc,"US action in Venezuela not legal, senior Labou...",https://www.bbc.com/news/articles/cwyr3q3xlqzo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2377/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,venezuela,66
6,trump,48
3,maduro,42
50,new,23
115,oil,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
219,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...,203
148,2026-01-05-04-57-44 +0000,nypost,"Sara Foster praises Trump’s capture of Maduro,...",https://nypost.com/2026/01/04/us-news/sara-fos...,177
122,2026-01-05-10-00-26 +0000,wapo,"What to know about Maduro, his capture and Tru...",https://www.washingtonpost.com/world/2026/01/0...,171
126,2026-01-05-09-20-28 +0000,wapo,Trump says U.S. is ‘in charge’ in Venezuela af...,https://www.washingtonpost.com/world/2026/01/0...,171
0,2026-01-05-16-53-53 +0000,nyt,Venezuela Live Updates: Maduro Arrives at Cour...,https://www.nytimes.com/live/2026/01/05/world/...,170


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
219,203,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...
45,54,2026-01-05-15-07-08 +0000,nypost,"Chevron, energy stocks soar after US capture o...",https://nypost.com/2026/01/05/business/chevron...
136,51,2026-01-05-07-55-46 +0000,nypost,Gov. Tim Walz expected to drop bid for re-elec...,https://nypost.com/2026/01/05/us-news/gov-tim-...
94,48,2026-01-05-12-00-00 +0000,wsj,Secretary of State Marco Rubio was there with ...,https://www.wsj.com/politics/elections/in-vene...
40,36,2026-01-05-15-15-57 +0000,nypost,"Sen. Mark Kelly at risk of demotion, cut in mi...",https://nypost.com/2026/01/05/us-news/sen-mark...
186,33,2026-01-04-23-19-44 +0000,nypost,Freed Israeli hostage details fear of being ma...,https://nypost.com/2026/01/04/world-news/freed...
35,32,2026-01-05-15-25-42 +0000,nypost,Paris court finds 10 people guilty of cyberbul...,https://nypost.com/2026/01/05/world-news/paris...
169,31,2026-01-05-01-05-44 +0000,nypost,Far-left extremist attack on Berlin power grid...,https://nypost.com/2026/01/04/world-news/far-l...
131,31,2026-01-05-08-51-15 +0000,bbc,South Korea's Lee Jae Myung meets China's Xi a...,https://www.bbc.com/news/articles/c70lw9x581no...
227,31,2026-01-04-19-53-22 +0000,nypost,Owner of Swiss nightclub where deadly fire bro...,https://nypost.com/2026/01/04/world-news/owner...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
